In [37]:
from sklearn.cross_validation import KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation, metrics 
from sklearn.grid_search import GridSearchCV 

/Users/lizida/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [1]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [2]:
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    

In [4]:
df= pd.read_csv('train_new3.csv')
test = pd.read_csv('test_new3.csv')

ntrain = df.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)


X_1= df[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]
y_1=df[['casual_log']]

test_1= test[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]

X_cas = X_1.values
y_cas = y_1.values
test_cas = test_1.values
rf_params = {
    'n_estimators': 900,
     'max_features': 8,
    'min_samples_leaf': 8,
    'min_samples_split':4,
}

rf = SklearnHelper(clf=RandomForestRegressor, seed=SEED, params=rf_params)
rf_oof_train, rf_oof_test = get_oof(rf,X_cas,y_cas, test_cas)

oof_rf_train = pd.DataFrame(rf_oof_train, columns=['casual_log'])
oof_rf_train.to_csv('oof_rf_train_log_casual.csv', index=False)
oof_rf_test = pd.DataFrame(rf_oof_test, columns=['casual_log'])
oof_rf_test.to_csv('oof_rf_test_log_casual.csv', index=False)

/Users/lizida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/lizida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/lizida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/lizida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/lizida/anacon

In [6]:
df.columns

Index(['Unnamed: 0', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'year', 'month', 'day', 'hour', 'weekday', 'windspeed_forest',
       'windspeed_linear', 'temp_scale', 'atemp_scale', 'humidity_scale',
       'windspeed_scale', 'windspeed_forest_scale', 'windspeed_linear_scale',
       'casual_log', 'registered_log', 'temp_humidity', 'temp_season',
       'humidity_season'],
      dtype='object')

In [12]:
X_register= df[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']].values
y_register=df[['registered_log']].values

test_1= test[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']].values


rf_params = {
    'n_estimators': 900,
     'max_features': 6,
    'min_samples_leaf': 6,
    'min_samples_split':2,
}

rf = SklearnHelper(clf=RandomForestRegressor, seed=SEED, params=rf_params)
rf_oof_train, rf_oof_test = get_oof(rf,X_register,y_register, test_1)

oof_rf_train = pd.DataFrame(rf_oof_train, columns=['register_log'])
oof_rf_train.to_csv('oof_rf_train_log_register.csv', index=False)
oof_rf_test = pd.DataFrame(rf_oof_test, columns=['register_log'])
oof_rf_test.to_csv('oof_rf_test_log_register.csv', index=False)

/Users/lizida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/lizida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/lizida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/lizida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/Users/lizida/anacon

In [147]:
X_1= df[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]
X_cas_gb = X_1.values
y_1=df[['casual_log']]
y_cas_gb = y_1.values
test_1= test[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]
test_cas_gb = test_1.values
gb_params = {
    'n_estimators': 450,
    'min_samples_leaf': 8,
    'min_samples_split':28,
}

gb = SklearnHelper(clf=GradientBoostingRegressor, seed=SEED, params=gb_params)
gb_oof_train, gb_oof_test = get_oof(gb,X_cas_gb,y_cas_gb, test_cas_gb)

oof_gb_train = pd.DataFrame(gb_oof_train, columns=['cas_log'])
oof_gb_train.to_csv('oof_gb_train_cas_log_sma_fea.csv', index=False)
oof_gb_test = pd.DataFrame(gb_oof_test, columns=['cas_log'])
oof_gb_test.to_csv('oof_gb_test_log_cas_sma_fea.csv', index=False)

/Users/lizida/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [148]:
X_1= df[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]
X_reg_gb = X_1.values
y_1=df[['registered_log']]
y_reg_gb = y_1.values
test_1= test[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]
test_reg_gb = test_1.values
gb_params = {
    'n_estimators': 700,
    'min_samples_leaf': 6,
    'min_samples_split':4,
}
from sklearn.ensemble import RandomForestRegressor
gb = SklearnHelper(clf=GradientBoostingRegressor, seed=SEED, params=gb_params)
gb_oof_train, gb_oof_test = get_oof(gb,X_reg_gb,y_reg_gb, test_reg_gb)

oof_gb_train = pd.DataFrame(gb_oof_train, columns=['reg_log'])
oof_gb_train.to_csv('oof_gb_train_reg_log_sma_fea.csv', index=False)
oof_gb_test = pd.DataFrame(gb_oof_test, columns=['reg_log'])
oof_gb_test.to_csv('oof_gb_test_log_reg_sma_fea.csv', index=False)

/Users/lizida/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [244]:
df_cas_gb=pd.read_csv('oof_gb_train_cas_log_sma_fea.csv')
df_cas_rf=pd.read_csv('oof_rf_train_log_casual.csv')
x_train_cas = np.concatenate(( df_cas_gb.values, df_cas_rf.values), axis=1)
y_train_cas=df[['casual_log']]

df_cas_gb_te=pd.read_csv('oof_gb_test_log_cas_sma_fea.csv')
df_cas_rf_te=pd.read_csv('oof_rf_test_log_casual.csv')
x_test_cas = np.concatenate(( df_cas_gb_te.values, df_cas_rf_te.values), axis=1)

In [245]:
df_reg_gb=pd.read_csv('oof_gb_train_reg_log_sma_fea.csv')
df_reg_rf=pd.read_csv('oof_rf_train_log_register.csv')

x_train_reg = np.concatenate(( df_reg_gb.values,df_reg_rf.values), axis=1)

y_train_reg = df[['registered_log']].values

df_reg_gb_te=pd.read_csv('oof_gb_test_log_reg_sma_fea.csv')
df_reg_rf_te=pd.read_csv('oof_rf_test_log_register.csv')
x_test_reg = np.concatenate(( df_reg_gb_te.values, df_reg_rf_te.values), axis=1)

In [154]:
xgb_0_cas = XGBRegressor()
xgb_0_cas.fit(x_train_cas,y_train_cas)
pred_0_cas=np.exp(xgb_0_cas.predict(x_test_cas))

xgb_0_reg = XGBRegressor()
xgb_0_reg.fit(x_train_reg,y_train_reg)
pred_0_reg=np.exp(xgb_0_reg.predict(x_test_reg))

pred = pred_0_cas+pred_0_reg-2
id_1 =pd.read_csv('test_bike.csv')
id_1['count'] = pred
sub = id_1[['datetime','count']]
sub.to_csv("sub4.csv", index=False)
## 0.37102

In [128]:
def rmsle(y, y_):
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return (sum(calc) * (1.0/len(y))) ** 0.5


rmsle_score = make_scorer(rmsle, greater_is_better=False)

# cas 调优

## 

In [151]:
xgb_param_grid = {"n_estimators":[100]}
grid = GridSearchCV(XGBRegressor(random_state = 10),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_cas, y_train_cas)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.44906, std: 0.05123, params: {'n_estimators': 100}],
 {'n_estimators': 100},
 -0.4490570153622867)

In [46]:
xgb_param_grid = {"n_estimators":[93,94,95]}
grid = GridSearchCV(XGBRegressor(random_state = 10),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_cas, y_train_cas)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.44888, std: 0.05135, params: {'n_estimators': 93},
  mean: -0.44882, std: 0.05123, params: {'n_estimators': 94},
  mean: -0.44886, std: 0.05121, params: {'n_estimators': 95}],
 {'n_estimators': 94},
 -0.44882262913284965)

In [158]:
xgb_param_grid = {"learning_rate":[0.1,0.099]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=94),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_cas, y_train_cas)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.44882, std: 0.05123, params: {'learning_rate': 0.1},
  mean: -0.44924, std: 0.05189, params: {'learning_rate': 0.099}],
 {'learning_rate': 0.1},
 -0.44882262913284965)

In [168]:
xgb_param_grid = {'subsample':[0.9,0.91,1]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=94),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_cas, y_train_cas)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.44797, std: 0.05149, params: {'subsample': 0.9},
  mean: -0.44853, std: 0.05188, params: {'subsample': 0.91},
  mean: -0.44882, std: 0.05123, params: {'subsample': 1}],
 {'subsample': 0.9},
 -0.4479664999831851)

In [164]:
xgb_param_grid = {'colsample_bytree':[1,0.99]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=94,subsample=0.9),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_cas, y_train_cas)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.44797, std: 0.05149, params: {'colsample_bytree': 1},
  mean: -0.45393, std: 0.05462, params: {'colsample_bytree': 0.99}],
 {'colsample_bytree': 1},
 -0.4479664999831851)

In [80]:
xgb_param_grid = {'max_depth':[1,2,3,4]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=94,subsample=0.9),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_cas, y_train_cas)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.44883, std: 0.05104, params: {'max_depth': 1},
  mean: -0.44773, std: 0.05176, params: {'max_depth': 2},
  mean: -0.44797, std: 0.05149, params: {'max_depth': 3},
  mean: -0.45127, std: 0.05287, params: {'max_depth': 4}],
 {'max_depth': 2},
 -0.4477329691365875)

In [85]:
xgb_param_grid = {'min_child_weight':[25,26,27]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=94,subsample=0.9,max_depth = 2),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_cas, y_train_cas)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.44671, std: 0.05090, params: {'min_child_weight': 25},
  mean: -0.44664, std: 0.05090, params: {'min_child_weight': 26},
  mean: -0.44666, std: 0.05095, params: {'min_child_weight': 27}],
 {'min_child_weight': 26},
 -0.44664356484474743)

In [292]:
xgb_param_grid = {'reg_alpha':[0.99, 1, 1.01,0]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=94,subsample=0.9,max_depth = 2,min_child_weight=26),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_cas, y_train_cas)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.44654, std: 0.05072, params: {'reg_alpha': 0.99},
  mean: -0.44654, std: 0.05072, params: {'reg_alpha': 1},
  mean: -0.44659, std: 0.05069, params: {'reg_alpha': 1.01},
  mean: -0.44664, std: 0.05090, params: {'reg_alpha': 0}],
 {'reg_alpha': 1},
 -0.4465390499217911)

In [ ]:
XGBRegressor(random_state = 10,n_estimators=94,subsample=0.9,max_depth = 2,min_child_weight=26,reg_alpha=1)

# reg 调优

In [146]:
xgb_param_grid = {"n_estimators":[100]}
grid = GridSearchCV(XGBRegressor(random_state = 10),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.33231, std: 0.12935, params: {'n_estimators': 100}],
 {'n_estimators': 100},
 -0.33230649370239274)

In [ ]:
xgb_param_grid = {"n_estimators":[100]}
grid = GridSearchCV(XGBRegressor(random_state = 10),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {"n_estimators":[70]}
grid = GridSearchCV(XGBRegressor(random_state = 10),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {"learning_rate":[0.01,0.1]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {'subsample':[0.7]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {'colsample_bytree':[i/10.0 for i in range(1,10)]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {'max_depth':[1,2,3,4]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {'min_child_weight':[6,7]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7,max_depth=2),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {'reg_alpha':[0]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7,max_depth=2,min_child_weight=6),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

In [65]:
xgb_param_grid = {"n_estimators":[70]}
grid = GridSearchCV(XGBRegressor(random_state = 10),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.34109, std: 0.13448, params: {'n_estimators': 70}],
 {'n_estimators': 70},
 -0.3410926077646359)

In [67]:
xgb_param_grid = {"learning_rate":[0.01,0.1]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -1.94522, std: 0.30965, params: {'learning_rate': 0.01},
  mean: -0.34109, std: 0.13448, params: {'learning_rate': 0.1}],
 {'learning_rate': 0.1},
 -0.3410926077646359)

In [72]:
xgb_param_grid = {'subsample':[0.7]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.34059, std: 0.13517, params: {'subsample': 0.7},
  mean: -0.34145, std: 0.13389, params: {'subsample': 0.8}],
 {'subsample': 0.7},
 -0.340590021999753)

In [200]:
xgb_param_grid = {'colsample_bytree':[i/10.0 for i in range(1,10)]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.36364, std: 0.14041, params: {'colsample_bytree': 0.1},
  mean: -0.36364, std: 0.14041, params: {'colsample_bytree': 0.2},
  mean: -0.36364, std: 0.14041, params: {'colsample_bytree': 0.3},
  mean: -0.36364, std: 0.14041, params: {'colsample_bytree': 0.4},
  mean: -0.36364, std: 0.14041, params: {'colsample_bytree': 0.5},
  mean: -0.36364, std: 0.14041, params: {'colsample_bytree': 0.6},
  mean: -0.34334, std: 0.13677, params: {'colsample_bytree': 0.7},
  mean: -0.34334, std: 0.13677, params: {'colsample_bytree': 0.8},
  mean: -0.34334, std: 0.13677, params: {'colsample_bytree': 0.9}],
 {'colsample_bytree': 0.7},
 -0.343337243310164)

In [88]:
xgb_param_grid = {'max_depth':[1,2,3,4]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.34737, std: 0.13693, params: {'max_depth': 1},
  mean: -0.34032, std: 0.13501, params: {'max_depth': 2},
  mean: -0.34059, std: 0.13517, params: {'max_depth': 3},
  mean: -0.34284, std: 0.13288, params: {'max_depth': 4}],
 {'max_depth': 2},
 -0.340317116448831)

In [93]:
xgb_param_grid = {'min_child_weight':[6,7]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7,max_depth=2),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.34023, std: 0.13481, params: {'min_child_weight': 6},
  mean: -0.34029, std: 0.13489, params: {'min_child_weight': 7}],
 {'min_child_weight': 6},
 -0.3402299371108927)

In [205]:
grid

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=6, missing=None, n_estimators=70,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=10,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.7),
       fit_params={}, iid=True, n_jobs=1, param_grid={'reg_alpha': [0]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_absolute_error', verbose=0)

In [207]:
xgb_param_grid = {'reg_alpha':[0]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7,max_depth=2,min_child_weight=6),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.34075, std: 0.13585, params: {'reg_alpha': 0}],
 {'reg_alpha': 0},
 -0.340749894690095)

In [280]:
xgb_param_grid = {'gamma':[0,1]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7,max_depth=2,min_child_weight=6),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.34023, std: 0.13481, params: {'gamma': 0},
  mean: -0.34052, std: 0.13531, params: {'gamma': 1}],
 {'gamma': 0},
 -0.3402299371108927)

In [288]:

xgb_param_grid = {'colsample_bylevel':[0.2,1,0.9]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7,max_depth=2,min_child_weight=6),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.35884, std: 0.14571, params: {'colsample_bylevel': 0.2},
  mean: -0.34023, std: 0.13481, params: {'colsample_bylevel': 1},
  mean: -0.35884, std: 0.14571, params: {'colsample_bylevel': 0.9}],
 {'colsample_bylevel': 1},
 -0.3402299371108927)

In [291]:
xgb_param_grid = {'reg_lambda':[0.2,1,0.9,1.1]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7,max_depth=2,min_child_weight=6),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.34070, std: 0.13503, params: {'reg_lambda': 0.2},
  mean: -0.34023, std: 0.13481, params: {'reg_lambda': 1},
  mean: -0.34027, std: 0.13492, params: {'reg_lambda': 0.9},
  mean: -0.34032, std: 0.13491, params: {'reg_lambda': 1.1}],
 {'reg_lambda': 1},
 -0.3402299371108927)

# 调优之后预测

In [281]:
xgb_0_cas = XGBRegressor(random_state = 10,n_estimators=94,subsample=0.9,max_depth = 2,min_child_weight=26,reg_alpha=1)
xgb_0_cas.fit(x_train_cas,y_train_cas)
pred_0_cas=np.exp(xgb_0_cas.predict(x_test_cas))

xgb_0_reg = XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7,max_depth=2,min_child_weight=6)
xgb_0_reg.fit(x_train_reg,y_train_reg)
pred_0_reg=np.exp(xgb_0_reg.predict(x_test_reg))

pred = pred_0_cas+pred_0_reg-2
id_1 =pd.read_csv('test_bike.csv')
id_1['count'] = pred
sub = id_1[['datetime','count']]
sub.to_csv("sub5.csv", index=False)
##0.36982
##-1.5 是0.36794

# 在第二层增加一个新的特征，并且在无调优的情况下预测

In [185]:
df_cas_gb=pd.read_csv('oof_gb_train_cas_log_sma_fea.csv')
df_cas_rf=pd.read_csv('oof_rf_train_log_casual.csv')
x_train_cas = np.concatenate(( df_cas_gb.values, df_cas_rf.values), axis=1)
temp = x_train_cas[:,0]+x_train_cas[:,1]
data = {'x':x_train_cas[:,0],'y':x_train_cas[:,1],'z':temp}
data_fra=pd.DataFrame(data)
x_train_cas = data_fra.as_matrix()

y_train_cas=df[['casual_log']]

df_cas_gb_te=pd.read_csv('oof_gb_test_log_cas_sma_fea.csv')
df_cas_rf_te=pd.read_csv('oof_rf_test_log_casual.csv')
x_test_cas = np.concatenate(( df_cas_gb_te.values, df_cas_rf_te.values), axis=1)

temp = x_test_cas[:,0]+x_test_cas[:,1]
data = {'x':x_test_cas[:,0],'y':x_test_cas[:,1],'z':temp}
data_fra=pd.DataFrame(data)
x_test_cas = data_fra.as_matrix()



In [186]:
df_reg_gb=pd.read_csv('oof_gb_train_reg_log_sma_fea.csv')
df_reg_rf=pd.read_csv('oof_rf_train_log_register.csv')

x_train_reg = np.concatenate(( df_reg_gb.values,df_reg_rf.values), axis=1)
temp = x_train_reg[:,0]+x_train_reg[:,1]
data = {'x':x_train_reg[:,0],'y':x_train_reg[:,1],'z':temp}
data_fra=pd.DataFrame(data)
x_train_reg = data_fra.as_matrix()



y_train_reg = df[['registered_log']].values

df_reg_gb_te=pd.read_csv('oof_gb_test_log_reg_sma_fea.csv')
df_reg_rf_te=pd.read_csv('oof_rf_test_log_register.csv')
x_test_reg = np.concatenate(( df_reg_gb_te.values, df_reg_rf_te.values), axis=1)

temp = x_test_reg[:,0]+x_test_reg[:,1]
data = {'x':x_test_reg[:,0],'y':x_test_reg[:,1],'z':temp}
data_fra=pd.DataFrame(data)
x_test_reg= data_fra.as_matrix()

In [190]:
xgb_0_cas = XGBRegressor()
xgb_0_cas.fit(x_train_cas,y_train_cas)
pred_0_cas=np.exp(xgb_0_cas.predict(x_test_cas))

xgb_0_reg = XGBRegressor()
xgb_0_reg.fit(x_train_reg,y_train_reg)
pred_0_reg=np.exp(xgb_0_reg.predict(x_test_reg))

pred = pred_0_cas+pred_0_reg-1.8
id_1 =pd.read_csv('test_bike.csv')
id_1['count'] = pred
sub = id_1[['datetime','count']]
sub.to_csv("sub4_1.csv", index=False)
##0.37110

# 使用新的错误函数调优

In [212]:
def rmsle(y, y_):
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

rmsle_score = make_scorer(rmsle, greater_is_better=False)



## cas预测

In [259]:
xgb_param_grid = {"n_estimators":[11,12]}
grid = GridSearchCV(XGBRegressor(random_state = 10),
                    param_grid=xgb_param_grid, cv=5, scoring=rmsle_score)
grid.fit(x_train_cas, y_train_cas.values)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.61390, std: 0.02140, params: {'n_estimators': 11},
  mean: -0.61406, std: 0.02265, params: {'n_estimators': 12}],
 {'n_estimators': 11},
 -0.6138987205458895)

In [263]:
xgb_param_grid = {"learning_rate":[0.11,0.1,0.099,0.09]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=11),
                    param_grid=xgb_param_grid, cv=5, scoring=rmsle_score)
grid.fit(x_train_cas, y_train_cas.values)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.61386, std: 0.02312, params: {'learning_rate': 0.11},
  mean: -0.61390, std: 0.02140, params: {'learning_rate': 0.1},
  mean: -0.61386, std: 0.02143, params: {'learning_rate': 0.099},
  mean: -0.61467, std: 0.02025, params: {'learning_rate': 0.09}],
 {'learning_rate': 0.099},
 -0.6138554389585927)

In [267]:
xgb_param_grid = {'subsample':[0.8,0.9,0.91,1]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=11,learning_rate=0.099),
                    param_grid=xgb_param_grid, cv=5, scoring=rmsle_score)
grid.fit(x_train_cas, y_train_cas.values)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.61411, std: 0.02135, params: {'subsample': 0.8},
  mean: -0.61415, std: 0.02104, params: {'subsample': 0.9},
  mean: -0.61398, std: 0.02123, params: {'subsample': 0.91},
  mean: -0.61386, std: 0.02143, params: {'subsample': 1}],
 {'subsample': 1},
 -0.6138554389585927)

In [270]:
xgb_param_grid = {'colsample_bytree':[1,0.99,0.98]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=11,learning_rate=0.099),
                    param_grid=xgb_param_grid, cv=5, scoring=rmsle_score)
grid.fit(x_train_cas, y_train_cas.values)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.61386, std: 0.02143, params: {'colsample_bytree': 1},
  mean: -0.61356, std: 0.02167, params: {'colsample_bytree': 0.99},
  mean: -0.61356, std: 0.02167, params: {'colsample_bytree': 0.98}],
 {'colsample_bytree': 0.99},
 -0.61356248336459)

In [274]:
xgb_param_grid = {'max_depth':[0,1,2,3,4]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=11,subsample=1,colsample_bytree=1,learning_rate=0.099),
                    param_grid=xgb_param_grid, cv=5, scoring=rmsle_score)
grid.fit(x_train_cas, y_train_cas.values)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.53189, std: 0.02260, params: {'max_depth': 0},
  mean: -0.57361, std: 0.01843, params: {'max_depth': 1},
  mean: -0.60532, std: 0.02001, params: {'max_depth': 2},
  mean: -0.61386, std: 0.02143, params: {'max_depth': 3},
  mean: -0.61577, std: 0.02310, params: {'max_depth': 4}],
 {'max_depth': 0},
 -0.5318915439435742)

In [278]:
xgb_param_grid = {'min_child_weight':[1,2,3,4]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=11,subsample=0.9,colsample_bytree=0.99,max_depth = 2),
                    param_grid=xgb_param_grid, cv=5, scoring=rmsle_score)
grid.fit(x_train_cas, y_train_cas.values)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.60497, std: 0.02059, params: {'min_child_weight': 1},
  mean: -0.60497, std: 0.02059, params: {'min_child_weight': 2},
  mean: -0.60497, std: 0.02059, params: {'min_child_weight': 3},
  mean: -0.60497, std: 0.02059, params: {'min_child_weight': 4}],
 {'min_child_weight': 1},
 -0.6049748839433924)

In [243]:
xgb_param_grid = {'reg_alpha':[ 1.01,1.11]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.9,max_depth = 2,colsample_bytree=0.99
                                 ,min_child_weight=32),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_cas, y_train_cas)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.44498, std: 0.05270, params: {'reg_alpha': 1.01},
  mean: -0.44504, std: 0.05268, params: {'reg_alpha': 1.11}],
 {'reg_alpha': 1.01},
 -0.44498446892617927)

In [ ]:
XGBRegressor(random_state = 10,n_estimators=70,subsample=0.9,max_depth = 2,colsample_bytree=0.99
                                 ,min_child_weight=32,reg_alpha=1.01)

In [ ]:
xgb_param_grid = {"n_estimators":[100]}
grid = GridSearchCV(XGBRegressor(random_state = 10),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {"n_estimators":[70]}
grid = GridSearchCV(XGBRegressor(random_state = 10),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {"learning_rate":[0.01,0.1]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {'subsample':[0.7]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {'colsample_bytree':[i/10.0 for i in range(1,10)]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {'max_depth':[1,2,3,4]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {'min_child_weight':[6,7]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7,max_depth=2),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

xgb_param_grid = {'reg_alpha':[0]}
grid = GridSearchCV(XGBRegressor(random_state = 10,n_estimators=70,subsample=0.7,max_depth=2,min_child_weight=6),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_

## reg

In [ ]:
xgb_param_grid = {"n_estimators":[100]}
grid = GridSearchCV(XGBRegressor(random_state = 10),
                    param_grid=xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
grid.fit(x_train_reg, y_train_reg)
grid.grid_scores_, grid.best_params_, grid.best_score_



# 由于之前效果不是很好，考虑加入一个预测值 比较差的模型，lasso

In [98]:
from sklearn.linear_model import Lasso

In [134]:

X_1= df[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]
y_1=df[['casual_log']]

test_1= test[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]

X_cas_la = X_1.values
y_cas_la = y_1.values
test_cas_la = test_1.values
la_params = {
    'alpha':0.00075
}

la= SklearnHelper(clf=Lasso, seed=SEED, params=la_params)
la_oof_train, la_oof_test = get_oof(la,X_cas_la,y_cas_la, test_cas_la)

oof_la_train = pd.DataFrame(la_oof_train, columns=['casual_log'])
oof_la_train.to_csv('oof_la_train_log_casual.csv', index=False)
oof_la_test = pd.DataFrame(la_oof_test, columns=['casual_log'])
oof_la_test.to_csv('oof_la_test_log_casual.csv', index=False)

In [135]:

X_1= df[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]
y_1=df[['registered_log']]

test_1= test[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]

X_reg_la = X_1.values
y_reg_la = y_1.values
test_reg_la = test_1.values
la_params = {
    'alpha':0.00075
}

la= SklearnHelper(clf=Lasso, seed=SEED, params=la_params)
la_oof_train, la_oof_test = get_oof(la,X_reg_la,y_reg_la, test_reg_la)

oof_la_train = pd.DataFrame(la_oof_train, columns=['reg_log'])
oof_la_train.to_csv('oof_la_train_log_reg.csv', index=False)
oof_la_test = pd.DataFrame(la_oof_test, columns=['reg_log'])
oof_la_test.to_csv('oof_la_test_log_reg.csv', index=False)

In [136]:
df_cas_gb=pd.read_csv('oof_gb_train_cas_log_sma_fea.csv')
df_cas_rf=pd.read_csv('oof_rf_train_log_casual.csv')
df_cas_la = pd.read_csv('oof_la_train_log_casual.csv')
x_train_cas = np.concatenate(( df_cas_gb.values, df_cas_rf.values,df_cas_la.values), axis=1)
y_train_cas=df[['casual_log']]

df_cas_gb_te=pd.read_csv('oof_gb_test_log_cas_sma_fea.csv')
df_cas_rf_te=pd.read_csv('oof_rf_test_log_casual.csv')
df_cas_la_te=pd.read_csv('oof_la_test_log_casual.csv')
x_test_cas = np.concatenate(( df_cas_gb_te.values, df_cas_rf_te.values,df_cas_la_te.values), axis=1)

In [137]:
df_reg_gb=pd.read_csv('oof_gb_train_reg_log_sma_fea.csv')
df_reg_rf=pd.read_csv('oof_rf_train_log_register.csv')
df_reg_la = pd.read_csv('oof_la_train_log_reg.csv')

x_train_reg = np.concatenate(( df_reg_gb.values,df_reg_rf.values,df_reg_la.values), axis=1)

y_train_reg = df[['registered_log']].values

df_reg_gb_te=pd.read_csv('oof_gb_test_log_reg_sma_fea.csv')
df_reg_rf_te=pd.read_csv('oof_rf_test_log_register.csv')
de_reg_la_te = pd.read_csv('oof_la_test_log_reg.csv')
x_test_reg = np.concatenate(( df_reg_gb_te.values, df_reg_rf_te.values,de_reg_la_te.values), axis=1)

## 不调优直接预测看一下结果

In [138]:
xgb_0_cas = XGBRegressor()
xgb_0_cas.fit(x_train_cas,y_train_cas)
pred_0_cas=np.exp(xgb_0_cas.predict(x_test_cas))

xgb_0_reg = XGBRegressor()
xgb_0_reg.fit(x_train_reg,y_train_reg)
pred_0_reg=np.exp(xgb_0_reg.predict(x_test_reg))

pred = pred_0_cas+pred_0_reg-2
id_1 =pd.read_csv('test_bike.csv')
id_1['count'] = pred
sub = id_1[['datetime','count']]
sub.to_csv("sub444.csv", index=False)
##结果是0.38164

# 在加入一个ridge

In [139]:
from sklearn.linear_model import Ridge

In [140]:
X_1= df[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]
y_1=df[['casual_log']]

test_1= test[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]

X_cas_dge = X_1.values
y_cas_dge = y_1.values.ravel()
test_cas_dge = test_1.values
la_params = {
    'alpha':0.1
}

dge= SklearnHelper(clf=Ridge, seed=SEED, params=la_params)
dge_oof_train, dge_oof_test = get_oof(dge,X_cas_dge,y_cas_dge, test_cas_dge)

oof_dge_train = pd.DataFrame(dge_oof_train, columns=['casual_log'])
oof_dge_train.to_csv('oof_dge_train_log_casual.csv', index=False)
oof_dge_test = pd.DataFrame(dge_oof_test, columns=['casual_log'])
oof_dge_test.to_csv('oof_dge_test_log_casual.csv', index=False)

In [141]:
X_1= df[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]
y_1=df[['registered_log']]

test_1= test[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed_forest', 'hour','weekday', 'year']]

X_reg_dge = X_1.values
y_reg_dge = y_1.values.ravel()
test_reg_dge = test_1.values
la_params = {
    'alpha':0.1
}

dge= SklearnHelper(clf=Ridge, seed=SEED, params=la_params)
dge_oof_train, dge_oof_test = get_oof(dge,X_reg_dge,y_reg_dge, test_reg_dge)

oof_dge_train = pd.DataFrame(dge_oof_train, columns=['reg_log'])
oof_dge_train.to_csv('oof_dge_train_log_reg.csv', index=False)
oof_dge_test = pd.DataFrame(dge_oof_test, columns=['reg_log'])
oof_dge_test.to_csv('oof_dge_test_log_reg.csv', index=False)

In [142]:
df_cas_gb=pd.read_csv('oof_gb_train_cas_log_sma_fea.csv')
df_cas_rf=pd.read_csv('oof_rf_train_log_casual.csv')
df_cas_la = pd.read_csv('oof_la_train_log_casual.csv')
df_cas_dge=pd.read_csv('oof_dge_train_log_casual.csv')
x_train_cas = np.concatenate(( df_cas_gb.values, df_cas_rf.values,df_cas_la.values,df_cas_dge.values), axis=1)
y_train_cas=df[['casual_log']]

df_cas_gb_te=pd.read_csv('oof_gb_test_log_cas_sma_fea.csv')
df_cas_rf_te=pd.read_csv('oof_rf_test_log_casual.csv')
df_cas_la_te=pd.read_csv('oof_la_test_log_casual.csv')
df_cas_dge_te=pd.read_csv('oof_dge_test_log_casual.csv')
x_test_cas = np.concatenate(( df_cas_gb_te.values, df_cas_rf_te.values,df_cas_la_te.values,df_cas_dge_te.values), axis=1)

In [143]:
df_reg_gb=pd.read_csv('oof_gb_train_reg_log_sma_fea.csv')
df_reg_rf=pd.read_csv('oof_rf_train_log_register.csv')
df_reg_la = pd.read_csv('oof_la_train_log_reg.csv')
df_reg_dge= pd.read_csv('oof_dge_train_log_reg.csv')

x_train_reg = np.concatenate(( df_reg_gb.values,df_reg_rf.values,df_reg_la.values,df_reg_dge.values), axis=1)

y_train_reg = df[['registered_log']].values

df_reg_gb_te=pd.read_csv('oof_gb_test_log_reg_sma_fea.csv')
df_reg_rf_te=pd.read_csv('oof_rf_test_log_register.csv')
de_reg_la_te = pd.read_csv('oof_la_test_log_reg.csv')
de_reg_dge_te=pd.read_csv('oof_dge_test_log_reg.csv')
x_test_reg = np.concatenate(( df_reg_gb_te.values, df_reg_rf_te.values,de_reg_la_te.values,de_reg_dge_te.values), axis=1)

In [144]:
xgb_0_cas = XGBRegressor()
xgb_0_cas.fit(x_train_cas,y_train_cas)
pred_0_cas=np.exp(xgb_0_cas.predict(x_test_cas))

xgb_0_reg = XGBRegressor()
xgb_0_reg.fit(x_train_reg,y_train_reg)
pred_0_reg=np.exp(xgb_0_reg.predict(x_test_reg))

pred = pred_0_cas+pred_0_reg-2
id_1 =pd.read_csv('test_bike.csv')
id_1['count'] = pred
sub = id_1[['datetime','count']]
sub.to_csv("sub555.csv", index=False)
##结果是0.38165